# Adiciona o Produto Interno Bruto e População dos Municípios à Matriz Origem-Destino

In [1]:
import pandas as pd

In [2]:
pib_municipal = pd.read_csv(
    'data/csv/pib_municipal.csv', 
    delimiter=';', 
    skiprows=3,   # Pular as primeiras 3 linhas
    nrows=5570    # Ler apenas 5570 linhas, de 4 até 5574
)

In [3]:
pop_municipal = pd.read_csv(
    'data/csv/pop_municipal.csv', 
    delimiter=';', 
    skiprows=3,   # Pular as primeiras 3 linhas
    nrows=5570    # Ler apenas 5570 linhas, de 4 até 5574
)

In [4]:
matriz_od_distancia = pd.read_csv('data/csv/matriz_od_distancia.csv', delimiter = ',')

In [5]:
matriz_od_distancia.columns

Index(['mun_origem', 'mun_destino', 'distancia'], dtype='object')

In [6]:
pop_municipal.columns

Index(['Cód.', 'Município', '2001', '2002', '2003', '2004', '2005', '2006',
       '2008', '2009', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021'],
      dtype='object')

In [7]:
pib_municipal.columns

Index(['Nível', 'Cód.', 'Município', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype='object')

In [8]:
def verificar_codigos(matriz_od_distancia, pib_municipal):
    mun_origem_unique = matriz_od_distancia['mun_origem'].unique()
    codigos_pib_municipal = pib_municipal['Cód.'].unique()

    # Encontrando os valores que não estão presentes
    codigos_nao_presentes = [mun for mun in mun_origem_unique if mun not in codigos_pib_municipal]
    
    return codigos_nao_presentes

In [9]:
# Executa a função e armazena os resultados
codigos_nao_presentes = verificar_codigos(matriz_od_distancia, pib_municipal)

# Exibe os códigos que não estão presentes
if codigos_nao_presentes:
    print(f"Os seguintes códigos de 'mun_origem' não estão presentes em 'Cód.': {codigos_nao_presentes}")
else:
    print("Todos os códigos de 'mun_origem' estão presentes em 'Cód.'")

Os seguintes códigos de 'mun_origem' não estão presentes em 'Cód.': [4300001, 4300002]


In [10]:
def criar_base_matriz_od_info_mun(matriz_od_distancia, pib_municipal, pop_municipal):
    # Fazer o merge da base matriz_od_distancia com os valores de PIB de origem
    matriz_od_info_mun = matriz_od_distancia.merge(
        pib_municipal[['Cód.', '2021']], 
        how='left', 
        left_on='mun_origem', 
        right_on='Cód.'
    ).rename(columns={'2021': 'pib_origem'})
    
    # Remover a coluna 'Cód.' resultante do merge
    matriz_od_info_mun.drop(columns=['Cód.'], inplace=True)

    # Fazer o merge novamente para adicionar os valores de PIB do destino
    matriz_od_info_mun = matriz_od_info_mun.merge(
        pib_municipal[['Cód.', '2021']], 
        how='left', 
        left_on='mun_destino', 
        right_on='Cód.'
    ).rename(columns={'2021': 'pib_destino'})
    
    # Remover a coluna 'Cód.' resultante do segundo merge
    matriz_od_info_mun.drop(columns=['Cód.'], inplace=True)

    # Fazer o merge da base matriz_od_info_mun com os valores de população de origem
    matriz_od_info_mun = matriz_od_info_mun.merge(
        pop_municipal[['Cód.', '2021']], 
        how='left', 
        left_on='mun_origem', 
        right_on='Cód.'
    ).rename(columns={'2021': 'pop_origem'})
    
    # Remover a coluna 'Cód.' resultante do merge
    matriz_od_info_mun.drop(columns=['Cód.'], inplace=True)

    # Fazer o merge novamente para adicionar os valores de população do destino
    matriz_od_info_mun = matriz_od_info_mun.merge(
        pop_municipal[['Cód.', '2021']], 
        how='left', 
        left_on='mun_destino', 
        right_on='Cód.'
    ).rename(columns={'2021': 'pop_destino'})
    
    # Remover a coluna 'Cód.' resultante do segundo merge
    matriz_od_info_mun.drop(columns=['Cód.'], inplace=True)

    # Remover linhas onde não há correspondência para PIB ou população de origem ou destino
    matriz_od_info_mun.dropna(subset=['pib_origem', 'pib_destino', 'pop_origem', 'pop_destino'], inplace=True)

    # Reindexar o DataFrame para ajustar o índice
    matriz_od_info_mun.reset_index(drop=True, inplace=True)

    return matriz_od_info_mun

In [11]:
matriz_od_info_mun = criar_base_matriz_od_info_mun(matriz_od_distancia, pib_municipal, pop_municipal)

In [12]:
# Salvando a matriz_od_info_mun como um arquivo CSV
output_path_csv = 'data/csv/matriz_od_info_mun.csv'
matriz_od_info_mun.to_csv(output_path_csv, index=False)